# 酷我音樂
- 下載酷我音樂平台上電台的「專輯」音檔

# 載入套件

In [1]:
import re
import os
import time
import requests
from bs4 import BeautifulSoup

# 設定爬蟲參數

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.63',
           'Cookie': '_ga=GA1.2.841063431.1603504850; Hm_lvt_cdb524f42f0ce19b169a8071123a4797=1611556993; _gid=GA1.2.997533225.1613194048; Hm_lpvt_cdb524f42f0ce19b169a8071123a4797=1613194071; kw_token=UCMBLA99FF',
           'referer': 'http://www.kuwo.cn/',
           'If-Range': '6023dfa9-8d7179',
           'Sec-Fetch-Dest': 'video',
           'Sec-Fetch-Mode': 'no-cors',
           'Sec-Fetch-Site': 'cross-site',
           'csrf': 'UCMBLA99FF'}

# 爬取資料

## 蒐集連結清單

In [3]:
page = 1
links = []

while True:
    # 生成網址
    url = 'http://www.kuwo.cn/api/www/album/albumInfo?albumId=547562&pn={}&rn=30'.format(page)
    
    # 請求資料
    resp = requests.get(url, headers=headers)
    time.sleep(0.5)
    try:
        musicList = [i['rid'] for i in resp.json()['data']['musicList']]

        # 保存資訊
        links += musicList

        # 輸出資料擷取進度
        print(page, ': ', len(list(set(links))))

        # 判斷是否跳出迴圈
        page += 1
        if len(musicList) < 30:
            links = list(set(links))
            print('There are totally {} links!'.format(len(links)))
            break
    except:
        print('status_code: ', resp.status_code, ', Retry')

1 :  30
2 :  60
status_code:  504 , Retry
status_code:  504 , Retry
status_code:  504 , Retry
3 :  90
4 :  120
status_code:  504 , Retry
5 :  150
6 :  180
7 :  210
8 :  240
9 :  270
10 :  300
11 :  330
12 :  360
13 :  390
14 :  420
status_code:  504 , Retry
15 :  450
16 :  480
17 :  510
18 :  540
19 :  570
status_code:  504 , Retry
20 :  600
21 :  630
22 :  660
23 :  690
24 :  720
status_code:  504 , Retry
25 :  750
26 :  780
27 :  810
28 :  840
status_code:  504 , Retry
29 :  870
30 :  900
31 :  915
There are totally 915 links!


# 下載連結音檔

In [4]:
# os.mkdir('./musics')

In [5]:
# 已下載的音樂清單
download_list = [int(i.split('_',-1)[0]) for i in os.listdir('./musics')]
len(download_list)

907

In [6]:
# 排除已下載的音樂
links = [link  for link in links if link not in download_list]
len(links)

8

In [7]:
for link in links:
    # 取音檔摘要
    url = 'http://www.kuwo.cn/play_detail/{}'.format(link)
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text)

    time.sleep(3)
    music_name = soup.find('title').text
    music_name = re.sub(r'/|', '', music_name)
    music_uploadtime = soup.find('span', {'class':'time'}).text
    
    # 取音檔連結
    music_link = 'http://www.kuwo.cn/url?format=mp3&rid={}&response=url&type=convert_url3&br=128kmp3'.format(link)
    try:
        music_link = requests.get(music_link).json()['url']
    except:
        time.sleep(1)
        music_link = requests.get(music_link).json()['url']
    # 下載音檔
    music_content = requests.get(url=music_link).content
    with open('./musics/{}.mp3'.format(str(link) + '_' + music_name), 'wb') as f:
        f.write(music_content)
        print('Succed: ', link, music_name)

Succed:  7090145 请你不要告诉我，你最爱的人不是我(一个人听)_蕊希Erin_单曲在线试听_酷我音乐
Succed:  7090146 为什么，你就不喜欢我了(一个人听)_蕊希Erin_单曲在线试听_酷我音乐
Succed:  76240867 ”我不是傻，只是懒得计较”_蕊希Erin_单曲在线试听_酷我音乐
Succed:  83795941 你有多久没对另一半说我爱你了？_蕊希Erin_单曲在线试听_酷我音乐
Succed:  57690089 会有人拿着戒指对你笑，说余生请多指教。 (节目)_蕊希Erin_单曲在线试听_酷我音乐
Succed:  57151470 ” 我月薪三万，拒绝给女友买两万的包。 ”_蕊希__单曲在线试听_酷我音乐
Succed:  148090871 总有人在偷偷羡慕你_蕊希Erin_单曲在线试听_酷我音乐
Succed:  41351166 “我们再也不会互道晚安”_蕊希Erin_单曲在线试听_酷我音乐


# 異常狀況排除
- 通常是檔案名稱有非法字元，或者request的速度過快被擋~

In [9]:
soup.find('title').text

'蕊希专访 | 杜海涛：爱你这件事，不是说说而已_蕊希Erin_单曲在线试听_酷我音乐'

In [25]:
requests.get(music_link).json()['url']

'https://sj-sycdn.kuwo.cn/0d9d21422d52d1dc95e2633821083e78/6027939d/resource/n1/76/94/3229346901.mp3'

In [16]:
music_name = '66823804_蕊希专访 陈乔恩：爱自己，是终身浪漫的开始_蕊希Erin_单曲在线试听_酷我音乐.mp3'

In [17]:
with open('./musics/{}.mp3'.format(str(link) + '_' + music_name), 'wb') as f:
    f.write(music_content)